In [ ]:
from __future__ import print_function
import sys
sys.path.append('deps/sparse_rrt')

import model.AE.identity as cae_identity
from model.AE import CAE_acrobot_voxel_2d
from model import mlp, mlp_acrobot
#from model.mlp import MLP
from model.mpnet import KMPNet
import numpy as np
import argparse
import os
import torch
from gem_eval import eval_tasks
from torch.autograd import Variable
import copy
import os
import gc
import random
from tools.utility import *
from plan_utility import pendulum, acrobot_obs
#from sparse_rrt.systems import standard_cpp_systems
#from sparse_rrt import _sst_module
from tools import data_loader
import jax
import matplotlib
#matplotlib.use("TkAgg")
import matplotlib.pyplot as plt


import torch.nn as nn
from torch.autograd import Variable
import math
import time
from sparse_rrt.systems import standard_cpp_systems
from sparse_rrt import _sst_module
from plan_utility.data_structure import *
#import matplotlib.pyplot as plt
#fig = plt.figure()

import sys
sys.path.append('..')

import numpy as np
#from tvlqr.python_tvlqr import tvlqr
#from tvlqr.python_lyapunov import sample_tv_verify
from plan_utility.data_structure import *
    

In [ ]:


def node_nearby(x0, x1, S, rho, system):
    # state x0 to state x1
    delta_x = x0 - x1
    circular = system.is_circular_topology()
    for i in range(len(delta_x)):
        if circular[i]:
            # if it is angle
            if delta_x[i] > np.pi:
                delta_x[i] = delta_x[i] - 2*np.pi
            if delta_x[i] < -np.pi:
                delta_x[i] = delta_x[i] + 2*np.pi

    xTSx = delta_x.T@S@delta_x
    if xTSx <= 4.:
        print('nearby:')
        print('S:')
        print(S)
        print('xTSx: %f' % (xTSx))
        print('rho^2: %f' % (rho*rho))
    if xTSx <= rho*rho:
         return True
    return False

def line_nearby(x0, x1, system):
    # state x0 to line starting from node x1
    e = x1.edge
    xs = e.xs
    us = e.us
    ts = e.time_knot
    for k in range(len(ts)-1):
        S = e.S(ts[k]).reshape((len(x0), len(x0)))
        rho = e.rho0s[k]
        if node_nearby(x0, xs[k], S, rho, system):
            return True, k
    return False, 0

def nearby(x0, x1, system):
    # using the S and rho stored by the node to determine distance
    # if x0 lies in x1, and within the boundary of x1 (S, rho0)
    # notice that for circulating state, needs to map the angle
    # if edge is defined on xG, then use edge
    if x0.edge is not None:
        e = x0.edge
        xs = e.xs
        us = e.us
        ts = e.time_knot
        for k in range(len(ts)-1):
            if x1.edge is not None:
                line_near, k1 = line_nearby(xs[k], x1, system)
                if line_near:
                    # near the line, with node index k1
                    return True, k, k1
            else:
                if node_nearby(xs[k], x1.x, x1.S0, x1.rho0, system):
                    return True, k, 0
    else:
        if x1.edge is not None:
            line_near, k1 = line_nearby(x0.x, x1, system)
            if line_near:
                return True, 0, k1
        else:
            if node_nearby(x0.x, x1.x, x1.S0, x1.rho0, system):
                return True, 0, 0
    return False, 0, 0

def node_h_nearby(x0, x1, S, rho, system):
    # given two nodes, S and rho, check if x0 is near x1
    delta_x = x0 - x1
    # this is pendulum specific. For other envs, need to do similar things
    circular = system.is_circular_topology()
    for i in range(len(delta_x)):
        if circular[i]:
            # if it is angle
            if delta_x[i] > np.pi:
                delta_x[i] = delta_x[i] - 2*np.pi
            if delta_x[i] < -np.pi:
                delta_x[i] = delta_x[i] + 2*np.pi
    xTSx = delta_x.T@S@delta_x
    if xTSx <= 4.:
        print('delta_x:')
        print(delta_x)
        print('S:')
        print(S)
        print('xTSx: %f' % (xTSx))
        # notice that we define rho to be ||S^{1/2}x||
        print('rho^2: %f' % (rho*rho))
    if xTSx <= rho*rho:
        return True
    else:
        return False
def line_h_nearby(x0, xG, S, rho, system):
    # check state x0 against line starting from node xG
    e2 = xG.edge
    xs2 = e2.xs
    us2 = e2.us
    ts2 = e2.time_knot
    for k2 in range(len(ts2)-1):
        if e2.S is not None:
            S = e2.S(ts2[k2]).reshape((len(x0), len(x0)))
            rho = e2.rho0s[k2]
        elif xG.S0 is not None:
            S = xG.S0
            rho = xG.rho0
        if node_h_nearby(x0, xs2[k2], S, rho, system):
            return True
    return False
def h_nearby(node, xG, S, rho, system):
    # check if the edge starting from node and the edge starting from xG has intersection
    if node.edge is not None:
        e1 = node.edge
        xs1 = e1.xs
        us1 = e1.us
        ts1 = e1.time_knot
        for k1 in range(len(ts1)-1):
            if xG.edge is not None:
                if line_h_nearby(xs1[k1], xG, S, rho, system):
                    return True
            else:
                if xG.S0 is not None:
                    S = xG.S0
                    rho = xG.rho0
                if node_h_nearby(xs1[k1], xG.x, S, rho, system):
                    return True
    else:
        if xG.edge is not None:
            if line_h_nearby(node.x, xG, S, rho, system):
                return True
        else:
            if xG.S0 is not None:
                S = xG.S0
                rho = xG.rho0
            if node_h_nearby(node.x, xG.x, S, rho, system):
                return True            
    return False
     

In [ ]:
a0 = np.array([0.,0.])
a1 = np.array([4.,4.])
da = np.array([1.,1.])
u = np.array([0.])

# a0 -> a1
x0 = Node(a0)
x1 = Node(a1)
x0.next = x1
xs = [a0, a0+da, a0+2*da, a0+3*da, a0+4*da]
us = [u, u, u, u, u]
dts = [0., 1., 2., 3., 4.]
time_knot = np.cumsum(dts)

e = Edge(xs, us, dts, time_knot, time_knot[-1])
e.next = x1
x0.edge = e
x1.prev = x0


# another
a0 = np.array([4.,0.])
a1 = np.array([0.,4.])
da = np.array([-1.,1.])
u = np.array([0.])

# a0 -> a1
y0 = Node(a0)
y1 = Node(a1)
y0.next = y1
xs = [a0, a0+da, a0+2*da, a0+3*da, a0+4*da]
us = [u, u, u, u, u]
dts = [0., 1., 2., 3., 4.]
time_knot = np.cumsum(dts)

e = Edge(xs, us, dts, time_knot, time_knot[-1])
e.next = y1
y0.edge = e
y1.prev = y0



